In [1]:
ls

README.md                     images/
clean_group_merge_data.ipynb  merged_dist_time_data.csv
county_pop.csv                time.csv
district_data.csv


In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [3]:
dist_df=pd.read_csv("district_data.csv")
time_df=pd.read_csv("time.csv")
county_pop_df=pd.read_csv("county_pop.csv")

In [4]:
columns_to_keep_district=["District No.", 
                        "District Name", 
                        "Location Address", 
                        "Land Area in Square Miles", 
                        "Home County", 
                        'K-12 Enrollment Fall 2017', 
                        'Dropout Rate\n (%)', 
                        "Attendance Rate",
                        "Free & Reduced Lunch Eligibility Percentage", 
                        "Percent of Special Education Students", 
                        "Average Teacher Salary", 
                        "Teacher - Avg Yrs of Experience", 
                        "Teacher - % with Advanced Degree", 
                        "ACT Composite Score", 
                        "No. of Students Taking the ACT" ]

columns_to_keep_time=["District Number",
                      "School Name",
                      "Instructional Days",
                      "Instructional Minutes",
                      "Instructional Hours",
                      "Operating 4 Day"]

dist_data_keeping_columns=dist_df.loc[:,columns_to_keep_district]
time_data_keeping_columns=time_df.loc[:,columns_to_keep_time]

In [5]:
time_data_keeping_columns["4_day_bin"]=pd.Series(np.where(time_data_keeping_columns["Operating 4 Day"].values == 'Yes', 1, 0),
          time_data_keeping_columns.index)


In [6]:
grouped_time_df=time_data_keeping_columns.groupby(['District Number']).mean()
grouped_time_df["4_day_bin"].value_counts()

0.000000    113
1.000000     31
0.666667      1
0.800000      1
0.750000      1
0.875000      1
0.250000      1
Name: 4_day_bin, dtype: int64

In [7]:
new_df=dist_data_keeping_columns.join(grouped_time_df, on="District No.", how='left')

In [8]:
pop_dict=dict(zip(county_pop_df["County"],county_pop_df["Population"]))

#Get counties from population df to match counties from school district df. Map population stat onto school district data frame by county.
pop_dict['Mc Cook'] = pop_dict["McCook"]
del pop_dict["McCook"]
pop_dict["McPherson_Sanborn"] = pop_dict['Sanborn\xa0and\xa0McPherson']
del pop_dict['Sanborn\xa0and\xa0McPherson']
new_df.iloc[103,4]='Oglala Lakota'
new_df.iloc[[47,84,114,147],4]="McPherson_Sanborn"


new_df['county_population'] = new_df['Home County'].map(pop_dict)
new_df

,District No.,District Name,Location Address,Land Area in Square Miles,Home County,K-12 Enrollment Fall 2017,Dropout Rate (%),Attendance Rate,Free & Reduced Lunch Eligibility Percentage,Percent of Special Education Students,Average Teacher Salary,Teacher - Avg Yrs of Experience,Teacher - % with Advanced Degree,ACT Composite Score,No. of Students Taking the ACT,Instructional Days,Instructional Minutes,Instructional Hours,4_day_bin,county_population
0,6001,Aberdeen School District 06-1,"1224 3rd St S, Aberdeen, SD 57401",419.72,Brown,4502,0.019,0.9497,0.369,0.1649,"48,431",13.0,0.3789,21.6,219,173.818182,62757.727273,1045.962121,0.00,38637.0
1,58003,Agar-Blunt-Onida School District 58-3,"500 8th St, Onida, SD 57564",1223.78,Sully,250,0.009,0.9575,0.296,0.1720,"43,267",17.9,0.2164,24.3,11,169.000000,59326.000000,988.766665,0.00,1390.0
2,61001,Alcester-Hudson School District 61-1,"102 E 5th, Alcester, SD 57001",194.25,Union,309,0.007,0.9477,0.259,0.1896,"42,273",8.0,0.2018,20.4,16,170.000000,61809.666667,1030.161110,0.00,14929.0
3,11001,Andes Central School District 11-1,"400 School St, Lake Andes, SD 57356",204.29,Charles Mix,316,0.071,0.9405,NaN,0.1114,"46,041",13.1,0.3030,NaN,9,162.200000,64569.400000,1076.156666,0.00,9319.0
4,38001,Arlington School District 38-1,"306 S Main St, Arlington, SD 57212",231.55,Kingsbury,258,0.000,0.9662,0.174,0.1227,"45,078",15.7,0.3274,22.9,12,166.000000,60244.000000,1004.066665,0.00,5000.0
5,21001,Armour School District 21-1,"604 3rd St, Armour, SD 57313",129.51,Douglas,167,0.000,0.9719,0.281,0.1198,"41,821",13.5,0.2857,21.8,11,175.000000,66143.666667,1102.394443,0.00,2940.0
6,4001,Avon School District 04-1,"210 Pine St, Avon, SD 57315",179.46,Bon Homme,230,0.000,0.9708,0.374,0.1667,"43,629",18.0,0.2456,23.4,14,165.250000,60179.250000,1002.987500,0.00,6972.0
7,49001,Baltic School District 49-1,"1 Bulldog Ave, Baltic, SD 57003",54.28,Minnehaha,486,0.009,0.9696,0.158,0.1111,"45,786",13.1,0.4167,21.6,28,179.500000,63681.000000,1061.350000,0.00,183660.0
8,9001,Belle Fourche School District 09-1,"2305 13th Ave, Belle Fourche, SD 57717",956.93,Butte,1378,0.016,0.9524,0.459,0.1814,"44,128",12.0,0.3261,21.8,58,158.000000,62122.400000,1035.373334,0.80,10194.0
9,3001,Bennett County School District 03-1,"403 1st Ave, Martin, SD 57551",1190.93,Bennett,480,0.037,0.9169,0.783,0.1475,"44,788",14.2,0.1405,18.9,22,149.333333,62060.000000,1034.333333,1.00,3440.0


In [9]:
new_df["zip"]=[int(str[-5:]) for str in new_df["Location Address"]]

In [10]:
mask=new_df["zip"]==57532
new_df[mask]

,District No.,District Name,Location Address,Land Area in Square Miles,Home County,K-12 Enrollment Fall 2017,Dropout Rate (%),Attendance Rate,Free & Reduced Lunch Eligibility Percentage,Percent of Special Education Students,Average Teacher Salary,Teacher - Avg Yrs of Experience,Teacher - % with Advanced Degree,ACT Composite Score,No. of Students Taking the ACT,Instructional Days,Instructional Minutes,Instructional Hours,4_day_bin,county_population,zip
123,57001,Stanley County School District 57-1,"112 S 1st Ave, Fort Pierre, SD 57532",1516.58,Stanley,449,0.009,0.9554,0.344,0.1514,"42,518",14.1,0.2696,19.1,18,160.0,63463.5,1057.724998,1.0,2994.0,57532


In [11]:
new_df.to_csv("merged_dist_time_data.csv")

In [12]:
new_df.isna().sum(axis=0)

District No.                                    0
District Name                                   0
Location Address                                0
Land Area in Square Miles                       0
Home County                                     0
K-12 Enrollment Fall 2017                       0
Dropout Rate\n (%)                              0
Attendance Rate                                 0
Free & Reduced Lunch Eligibility Percentage    11
Percent of Special Education Students           1
Average Teacher Salary                          0
Teacher - Avg Yrs of Experience                 0
Teacher - % with Advanced Degree                0
ACT Composite Score                            41
No. of Students Taking the ACT                  0
Instructional Days                              0
Instructional Minutes                           0
Instructional Hours                             0
4_day_bin                                       0
county_population                               0
